## $\lambda$ calculus

Here  is a Python notebook to play with the lamdba calculus and  evaluate lambda expressions of the form `λvar1.(exp1) λvar2.(exp2) ...`. If you don't know Python you can safely ignore the Ptyhon code and skip below to where we actually talk about the $\lambda$ calculus itself.

One note is that the code is quite sensitive to spaces and parenthesis placements. In particular because, following [Matt Might](http://matt.might.net/articles/python-church-y-combinator/), we'll use Python's lambda expressions mechanisms, we'll need to wrap everything with extra parenthesis. 

You can  find better lambda evaluators online such as [this one](https://people.eecs.berkeley.edu/~gongliang13/lambda/) 

There are also some annoying issues due to the fact that Python is an _eager_ language which means that if we have a lambda expression such as $(\lambda x,y.x) (e)(f)$ then Python will evaluate $f$ even though it is not necessary for the computation. It is probably easy to avoid this (indeed maybe even using Matt Might's ideas above) but at the moment I just used a simple hack to handle this (which unfortuantely does not work for the Y combinator yet)

If you want to use the λ character you can copy paste it from here:  λ

In [1]:
# convert lambda expression to python code
# all it amounts to is replacing λ with lambda and . with :
def ltop(exp):
    return eval("("+exp.replace('λ','lambda ').replace('.',': ')+")")
        
# evaluate whether Python code  corresponds to 0 or 1
def boolp(code): 
    res = code("first")("second") 
    if res=='first': return 1
    if res=='second': return 0
    return code

OK now that we got Python out of the way, let us play with this and see if it works

In [2]:
ltop("(λx.(λy.(y)))  ('hello')('world')")

'world'

In [38]:
one  = ltop("λx.(λy.(x))")
zero = ltop("λx.(λy.(y))") 

In [39]:
one("first")("second")

'first'

Recall the following question from lecture:

We had the following question:

__Question:__ Suppose that $F = \lambda f. (\lambda x. (f x)f)$, $1 = \lambda x.(\lambda y.x)$ and $0=\lambda x.(\lambda y.y)$.
What is $F \; 1\; 0$?

__a.__ $1$

__b.__ $0$

__c.__ $\lambda x.1$

__d.__ $\lambda x.0$

Let's evaluate the answer

In [5]:
mystery=ltop("(λf.(λx.((f (x) (f)))))")

In [6]:
mystery (one)(zero)

<function __main__.<lambda>>

This is not very informative, but we can test if this is the zero function $\lambda x,y.y$ or the one function $\lambda y,x.y$ or something else

In [7]:
boolp(_)

0

In [8]:
NIL=ltop("λf.(one)")
PAIR =ltop("λx.(λy.(λf.((f (x)) (y))))")
ISEMPTY=ltop("λP.(P (λx.(λy.(zero))))")
HEAD = ltop("λP.(P (one))")
TAIL  =ltop("λP.(P (zero))")

In [9]:
boolp(ISEMPTY(NIL))

1

In [10]:
HEAD(PAIR("first")("second"))

'first'

Now let's try something a little more interesting

Recall that we defined

$$APAR \;\;\;  = \lambda f,L. (ISNIL \; L)\; 0 \; (XOR \; (HEAD \; L) \; (f \; f \; (TAIL \; L)) \color{green}{(**)}$$

and claimed that $APAR \; APAR$ (i.e., $APAR$ applied to itself) is the parity function. Let's try this out

The following is the definition of XOR - can you see why?

In [40]:
XOR = ltop("λa.(λb.(a (b (zero) (one)) (b)))")

In [12]:
boolp(XOR(one)(one))

0

In [41]:
APAR=ltop("λf.(λL.((ISEMPTY (L)) (zero) (XOR (HEAD (L) ) ((f (f)  ) (TAIL (L) )))))")

The code we used for `ltop` above works perfectly fine for non recursive functions. However, because Python is an "eager" programming language, we have to be a little more careful in programming functions that do not necessarily halt. Below is a very crude hack where we simply stop the recursion if it goes on too deep, with the hope that we'll never care about the result of this branch. If you have a simpler/better way to do it, then please let me know.

Please skip ahead and ignore this cell: this is about details that are more specific to Python than to the λ calculus.

If you actually run this notebook, you'd want to use this definition of `ltop` and rerun all the cells above using it.

In [327]:
# a little fancier version of ltop that keeps track of the lambda expression
# you can safely ignore this code
import inspect

from inspect import getouterframes, currentframe

MAX_RECURSION_DEPTH = 40


def ltop(exp):
    global MAX_RECURSION_DEPTH
    code = eval("("+exp.replace('λ','lambda ').replace('.',': ')+")")
    if not callable(code): return code
    def myfunc(x):
        # if len(getouterframes(currentframe())) > MAX_RECURSION_DEPTH: # chop off
        if len(inspect.stack()) > MAX_RECURSION_DEPTH:
            return lambda x: (lambda y: y)
        return code(x)
    myfunc.lambdaexp = exp
    return myfunc

The following will be slow because Python will go into unnecessary branches of the recursion.

In [42]:
par = APAR(APAR)

In [43]:
par(NIL)

((<function __main__.<lambda>>, <function __main__.ltop.<locals>.myfunc>),
 <function __main__.<lambda>>)

But at least we get the correct answer (0):

In [47]:
evaluate((APAR (APAR))(NIL)("first")("second"))

TypeError: 'tuple' object is not callable

Similarly we get the correct answers for the list $(1)$ and $(1,1)$, see below:

In [214]:
boolp(par (PAIR(one)(NIL)))

1

In [215]:
boolp(par (PAIR(one)(PAIR (one)(NIL) )))

0

The cells below correspond to a rough and still buggy part of the notebook. Feel free to ignore, and if you are a Python expert, feel free to tell me how to fix this.

Let's try to see if we can avoid some of these issues by defining IF which will do the "shortcut" evaluation 

In [284]:
IF = ltop("λf.(λx.(λy.((f (λignore.x) (λignore.y)) (zero)  )))") 
#lambda f: lambda x: lambda y: (f (lambda ignore: x) (lambda ignore: y))("nonsense")

In [285]:
IF(zero)("hello")("world")

'world'

In [286]:
def nothalt():
    x =0
    while True:
        x += 1

In [287]:
IF(one)("hello")(nothalt)

'hello'

In [288]:
APAR=ltop("λf.(λL.(IF (ISEMPTY (L)) (zero) (XOR (HEAD (L) ) ((f (f)  ) (TAIL (L) )))))")

In [289]:
 APAR(APAR)(NIL)

<function __main__.ltop.<locals>.myfunc>

In [290]:
boolp(_)

0

Wasn't any quicker but at least it's still correct. Let's now try to do the same thing via the Y combinator

In [291]:
PARREC = ltop("λf.(λL.((IF (ISEMPTY (L) ) (zero) (XOR (HEAD (L) ) (f (TAIL (L) ))))))")

In [292]:
Y = ltop("λf.((λx.(f (x (x)) )) (λy.(f (y (y)  ))))")

In [310]:
par = Y(PARREC)

RecursionError: maximum recursion depth exceeded

Well, might need a better hack for handling recursion .. if anyone has suggestions please let me know...

Please ignore the cells below, they are renmants from my attempt to evaluate the lambda expressions myself before I decided to use native Python

In [294]:
from inspect import getouterframes, currentframe # only for printing
# import os

import sys

sys.setrecursionlimit(3000)

OFFSET = len(getouterframes(currentframe()))    
    

def list_lambdas(exp):
    if not exp: return []
    exp = exp.strip()
    #print("Expression is:"+exp)
    if not exp: return []
    firstspace = exp.find(' ')
    begin = exp.find('(')
    if begin==-1 and firstspace==-1:
        #print(exp+"->"+exp)
        return [exp]
    if (begin==-1) or (firstspace>-1 and firstspace<begin):
        #print (exp + "->"+ str([exp[:firstspace]]+list_lambdas(exp[firstspace+1:])))
        return [exp[:firstspace]]+list_lambdas(exp[firstspace+1:])
    if begin>0 and exp[0]!='λ':
        return [exp[:begin]]+list_lambdas(exp[begin:])
    end = matchparen(exp,begin)
    #print  (exp +"->"+str([exp[:end+1]] +list_lambdas(exp[end+1:])))
    return [exp[:end+1]] +list_lambdas(exp[end+1:])
    

In [3]:
uniquectr =0 

def my_eval(exp,table={},debug=False):
    def getunique():
        global uniquectr
        uniquectr +=1
        return "u<"+str(uniquectr)+">"
    
    def pr(s):
        if debug: print(" "*(4*(len(getouterframes(currentframe()))-OFFSET))+s)
    # pr(exp)
    L = list_lambdas(exp)
    if len(L)==0: return ""
    if len(L)>1: 
        func = L[0]
        if is_lambda(func):
            e = beta_reduce(func,L[1],table,debug)
            if len(L)>2: return my_eval(e+" "+" ".join(L[2:]),table,debug)
            return e
        func =  my_eval(L[0],table,debug)
        if is_lambda(func):
            return my_eval(func+" "+" ".join(L[1:]),table,debug)
        return func+" "+" ".join([my_eval(tmp,table,debug) for tmp in L[1:]])
    exp = L[0]

    if exp in table:
        newtable = dict(table)
        del newtable[exp]
        pr(exp+"<-"+table[exp])
        return my_eval(table[exp],newtable,debug)

    if exp[0]=='(':
        return my_eval(is_paren(exp),table,debug)
    if is_lambda(exp):
        [var,subexp] = is_lambda(exp)
        newtable = dict(table)
        if var in newtable: del newtable[var]
        return "λ"+var+".("+my_eval(subexp,newtable,debug)+")"
    
    #if table:
     #   [myexp,newtable,found] = replacevars(exp,table)
     #   if found:
     #       return my_eval(myexp,newtable,debug)
    
        pr("Unparsed: "+exp)
    return exp

def beta_reduce(func,arg,table,debug):
        
    def pr(s):
        if debug: print(" "*(4*(len(getouterframes(currentframe()))-OFFSET))+s)
    
    if not is_lambda(func): raise Exception()
    
    [var,subexp] = is_lambda(func)
    newtable = dict(table)
    
    if len(list_lambdas(arg))>1:
        arg = "("+arg+")"
    newtable[var] = arg
    pr(subexp+" ["+var+"->"+arg+"]...")
    res =  my_eval(subexp,newtable,debug)
    pr(subexp+"["+var+"->"+arg+ "]  = "+ res)
    return res
    
def is_lambda(exp):
    if exp[0]!='λ': return False
    varend = exp.find('.')
    begin  = varend+1
    if exp[begin]=='(': 
        end = matchparen(exp,begin)
    else:
        begin = varend
        end=len(exp)
    var = exp[1:varend].strip()
    subexp = exp[begin+1:end]
    return [var,subexp] 

def is_paren(exp):
    if exp[0]!='(': return False
    end = matchparen(exp,0)
    return exp[1:end]

    
    

In [5]:
def matchparen(s,i):
    depth = 1
    if s[i]!='(': raise Exception()
    j = i+1
    while depth>0:
        if s[j]=='(': depth += 1
        if s[j]==')': depth -= 1
        j += 1
    if s[j-1]!=')': raise Exception()
    return j-1

In [6]:
def replacevars(s,table):
    newtable = dict(table)
    found = False
    for v in table:
        if v!=table[v] and s.find(v)>-1:
            s = s.replace(v,table[v])
            del newtable[v]
            found = True
    return [s,newtable,found]

In [7]:
my_eval("λx.(x+y) 7",debug=True)

            x+y [x->7]...
            x+y[x->7]  = x+y


'x+y'

We will now go over the examples in the lecture. We use `table` for our table of names.

We had the following question:

__Question:__ Suppose that $F = \lambda f. (\lambda x. (f x)f)$, $1 = \lambda x.(\lambda y.x)$ and $0=\lambda x.(\lambda y.y)$.
What is $F \; 1\; 0$?

__a.__ $1$

__b.__ $0$

__c.__ $\lambda x.1$

__d.__ $\lambda x.0$


We can answer this question by trying this out

In [8]:
table = {}

table["0"]="λx.(λy.(y))"
table["1"]="λx.(λy.(x))"
table["F"]="λf.(λx.((f x) f))"

In [9]:
my_eval("(F 1) 0",table,False)

'λx.(λy.(y))'

Which is the same as $0$

In [10]:
table = {}

table["0"]="λx.(λy.(y))"
table["1"]="λx.(λy.(x))"
table["NIL"]="λf.(1)"
table["PAIR"]="λx.(λy.(λf.((f x) y)))"
table["ISEMPTY"]="λP.(P (λx.(λy.(0))))"
table["HEAD"]="λP.(P 1)"
table["TAIL"]="λP.(P 0)"

In [11]:
my_eval("0 1 0",table,False)

'λx.(λy.(y))'

Now for the parity example.

Recall that we defined

$$APAR \;\;\;  = \lambda f,L. (ISNIL \; L)\; 0 \; (XOR \; (HEAD \; L) \; (f \; f \; (TAIL \; L)) \color{green}{(**)}$$

We first note that we can define XOR in the lambda calculus as follows

$$XOR = \lambda a,b.a(b 1 0)b$$

In [12]:
# table["XOR"]="λvar.(λb.(var (b 0 1) b))"
table["XOR"]="λvar.(λb.(var (b λz.(λw.w) λt.(λs.t)) b))"

In [13]:
my_eval("(XOR 1) 1",table,False)

'λt.(λs.(t))'

In [14]:
my_eval("(λb.(1 (b 0 1) b))",table)

'λb.(b λx.(λy.(y)) λx.(λy.(x)))'

In [15]:
my_eval("λhello.(hello λx.(λy.(x)) hello) 0",table)

'λx.(λy.(y))'

In [16]:
my_eval("(0 λx.(λy.(x)) 0)",table)

'λx.(λy.(y))'

In [17]:
table["APAR"]="λf.(λL.((ISEMPTY L) 0 (XOR (HEAD L) ((f f) (TAIL L)))))"


In [18]:
my_eval("TAIL (PAIR 1 0)",table)

'λx.(λy.(y))'

In [19]:
table

{'0': 'λx.(λy.(y))',
 '1': 'λx.(λy.(x))',
 'APAR': 'λf.(λL.((ISEMPTY L) 0 (XOR (HEAD L) ((f f) (TAIL L)))))',
 'HEAD': 'λP.(P 1)',
 'ISEMPTY': 'λP.(P (λx.(λy.(0))))',
 'NIL': 'λf.(1)',
 'PAIR': 'λx.(λy.(λf.((f x) y)))',
 'TAIL': 'λP.(P 0)',
 'XOR': 'λvar.(λb.(var (b λz.(λw.w) λt.(λs.t)) b))'}

In [20]:
my_eval("1",table)

'λx.(λy.(x))'

In [21]:
my_eval("ISEMPTY (PAIR 0 1) a b",table)

'b'

In [22]:
my_eval("APAR APAR",table,False)

'λL.(L λx.(λy.(λx.(λy.(y)))) λx.(λy.(y)) L λx.(λy.(x)) λf.(λL.(L λx.(λy.(λx.(λy.(y)))) λx.(λy.(y)) L λx.(λy.(x)) f f L λx.(λy.(y)) λz.(λw.(w)) λt.(λs.(t)) f f L λx.(λy.(y)))) λf.(λL.(L λx.(λy.(λx.(λy.(y)))) λx.(λy.(y)) L λx.(λy.(x)) f f L λx.(λy.(y)) λz.(λw.(w)) λt.(λs.(t)) f f L λx.(λy.(y)))) L λx.(λy.(y)) λz.(λw.(w)) λt.(λs.(t)) λf.(λL.(L λx.(λy.(λx.(λy.(y)))) λx.(λy.(y)) L λx.(λy.(x)) f f L λx.(λy.(y)) λz.(λw.(w)) λt.(λs.(t)) f f L λx.(λy.(y)))) λf.(λL.(L λx.(λy.(λx.(λy.(y)))) λx.(λy.(y)) L λx.(λy.(x)) f f L λx.(λy.(y)) λz.(λw.(w)) λt.(λs.(t)) f f L λx.(λy.(y)))) L λx.(λy.(y)))'

In [23]:
my_eval("(APAR APAR) NIL",table)

KeyboardInterrupt: 

In [24]:
my_eval("(APAR APAR) (PAIR 0 (PAIR 1 NIL))",table, False)

'λx.(λy.(y))'

In [ ]:
list_lambdas('f λx.(λy.(x))')

In [ ]:
eval_lambda("NIL",table)

In [25]:
table["YCOMB"] = "λf.((λx.(f (x x))) (λy.(f (y y))))"

In [26]:
table["REC"] = "λf.(λL.(((ISEMPTY L) 0) (XOR (HEAD L) (f (TAIL L)))))"

In [27]:
my_eval("YCOMB REC",table)

'λx.(λy.(y))'

In [28]:
my_eval("(XOR 0 0)",table)

'λx.(λy.(y))'

In [29]:
my_eval("TAIL (PAIR 7 (PAIR 9 (PAIR 11 12)))",table)

'λf.(f 9 λf.(f 11 12))'

In [30]:
my_eval("ISEMPTY (PAIR 0 1)",table)

'λx.(λy.(y))'

In [31]:
my_eval("(YCOMB REC) (PAIR 0 NIL)",table)

'λy.(y)'